#HW 10 CLUSTERING BUSINESS

###cluster time trends in NYC businesses: 
###DATA: Census Business data:
###download census data for businesses by ZIP code. the data is here
http://www.census.gov/econ/cbp/download/

###and it can be downloaded by hand. you can also download it with 3 terminal commands as follows: the data from 1993 through 2001 is different in the format of its path than the data after 2001 (that is why more than one for loop is needed)

$ for ((y=93; y<=99; y+=1)); do wget ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp$y\totals.zip; done

$ for ((y=0; y<=9; y+=1)); do wget ftp://ftp.census.gov/econ200$y\/CBP_CSV/zbp0$y\totals.zip; done

$ for ((y=10; y<=15; y+=1)); do wget ftp://ftp.census.gov/econ20$y\/CBP_CSV/zbp$y\totals.zip; done


#NYC zip codes shape file is here
##http://data.nycprepared.org/dataset/nyc-zip-code-tabulation-areas/resource/0c0e14e9-78e1-404e-97b0-c2fabceb3981


NOTES: 
to read in a zip file without unzipping it you can use the pandas and zipfile packages

In [1]:
#import zipfiles #i am not giving it a name cause i intend to use it only once
#zf = zipfile.ZipFile(fname)
#df = pd.read_csv(zf.open(fname.replace('.zip','.txt')))

you may need to clean your data: for some NYC zip codes there may be no info
    
sanity check: you should have 20 (Ntimestamps) datapoints per time series and about 250 zipcodes (Nzipcodes)

IMPORTANT: we talked about the importance of "whitening" your data: dividing each feature by its standard deviation. 
Whitenings decorrelates the data: it makes the features independent so that the data covariance matrix is the identity matrix.
Whitening your data in time series analysis is in most cases **wrong**: you are modifying your time behaviour. This is because of the strong correlation between features (two consecutive time stamps for the same observation, the same zip code here, are strongly correlated). Here instead you want to standardize your time series: subtract the mean and divide each time series (separately) by its standard deviation. As a sanity check (if you use skitlearn Kmeans or skitlearns kmeans2): you want your data array to be shaped Nzipcodes x Ntimestamps

mydata.shape should be (Nzipcodes, Ntimestamps)

mydata[i].std() shoould be 1 for all i in range(len(Nzipcodes))

mydata[i].mean() should be ~0 for all i in range(len(Nzipcodes))



TASKS:
    
    1. get and prep your data.
    2. cluster the time series with K-means in **a few** clusters (as discussed there is no real good, sound way to decide what a good number is here. try a few options, keeping in mind a few is more than a couple, but i recommand you stay within the single digit numbers)
    3. plot the cluster centers (if you used K means those are the means of the clusters). you can plot for example the cluster centers overlayed on each time series (using the alpha channel to control the opacity in the plot may be helpful here).
    4. Use another clustering algorithm (of your choice)
    5. overlay your data on a NYC map: you can use shapefiles for the zip codes and different colors for different clusters
    6. Compare the results of the 2 algorithms
    7. attempt an interpretation. this is dangerous ground: clustering is an exploratory tool so you do not want to jump to conclusions because you see some clusters! but seeing structure in your data can inform your next moves as an investigator. 
    

In [5]:
from __future__ import print_function
__author__ = 'mo'
import pylab as pl
import numpy as np
%pylab inline

import pandas as pd
import geopandas as gp
import json

import sklearn.cluster
from sklearn import mixture
from PIL import Image
from scipy.misc import imread
from scipy.cluster.vq import kmeans2
import os

kelly_colors_hex = [
    '#FFB300', # Vivid Yellow
    '#803E75', # Strong Purple
    '#FF6800', # Vivid Orange
    '#A6BDD7', # Very Light Blue
    '#C10020', # Vivid Red
    '#CEA262', # Grayish Yellow
    '#817066', # Medium Gray
    '#007D34', # Vivid Green
    '#F6768E', # Strong Purplish Pink
    '#00538A', # Strong Blue
    '#FF7A5C', # Strong Yellowish Pink
    '#53377A', # Strong Violet
    '#FF8E00', # Vivid Orange Yellow
    '#B32851', # Strong Purplish Red
    '#F4C800', # Vivid Greenish Yellow
    '#7F180D', # Strong Reddish Brown
    '#93AA00', # Vivid Yellowish Green
    '#593315', # Deep Yellowish Brown
    '#F13A13', # Vivid Reddish Orange
    '#232C16', # Dark Olive Green
    ]

#np.random.seed(123)

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [13]:
import os
print (os.environ['PUI2015'])
#"HOME" in os.environ

/Users/mariaortiz/PUI2015_mortiz/


In [17]:
#VERY unpythonic way of reading in data from csvs, and adding column for year
zf94 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp94totals.csv')
zf94['year'] = '1994'

zf95 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp95totals.csv')
zf95['year'] = '1995'

zf96 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp96totals.csv')
zf96['year'] = '1996'

zf97 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp97totals.csv')
zf97['year'] = '1997'

zf98 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp98totals.csv')
zf98['year'] = '1998'

zf99 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp99totals.csv')
zf99['year'] = '1999'

zf00 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp00totals.csv')
zf00['year'] = '2000'

zf01 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp01totals.csv')
zf01['year'] = '2001'

zf02 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp02totals.csv')
zf02['year'] = '2002'

zf03 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp03totals.csv')
zf03['year'] = '2003'

zf04 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp04totals.csv')
zf04['year'] = '2004'

zf05 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp05totals.csv')
zf05['year'] = '2005'

zf06 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp06totals.csv')
zf06['year'] = '2006'

zf07 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp07totals.csv')
zf07['year'] = '2007'

zf08 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp08totals.csv')
zf08['year'] = '2000'

zf09 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp09totals.csv')
zf09['year'] = '2009'

zf10 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp10totals.csv')
zf10['year'] = '2010'

zf11 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp11totals.csv')
zf11['year'] = '2011'

zf12 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp12totals.csv')
zf12['year'] = '2012'

zf13 = pd.read_csv(os.getenv('$PUI2015')+'/HW10/zbp13totals.csv')
zf13['year'] = '2013'

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [4]:
#sanity check and looking at data
#zf03.head()

In [16]:
#creating small dataframes for each year
df94 = zf94[['zip', 'est', 'year', 'name']]
df95 = zf95[['zip', 'est', 'year', 'name']]
df96 = zf96[['zip', 'est', 'year', 'name']]
df97 = zf97[['zip', 'est', 'year', 'name']]

#for some of these csvs, the headers change to caps, so again in very unpythonic way, changing the headers 
zf98['zip'] = zf98['ZIP']
zf98['est'] = zf98['EST']
zf98['name'] = zf98['NAME']
df98 = zf98[['zip', 'est', 'year', 'name']]

zf99['zip'] = zf99['ZIP']
zf99['est'] = zf99['EST']
zf99['name'] = zf99['NAME']
df99 = zf99[['zip', 'est', 'year', 'name']]

zf00['zip'] = zf00['ZIP']
zf00['est'] = zf00['EST']
zf00['name'] = zf00['NAME']
df00 = zf00[['zip', 'est', 'year', 'name']]

zf01['zip'] = zf01['ZIP']
zf01['est'] = zf01['EST']
zf01['name'] = zf01['NAME']
df01 = zf01[['zip', 'est', 'year', 'name']]

zf02['zip'] = zf02['ZIP']
zf02['est'] = zf02['EST']
zf02['name'] = zf02['NAME']
df02 = zf02[['zip', 'est', 'year', 'name']]

#lower case headers again
df03 = zf03[['zip', 'est', 'year', 'name']]
df04 = zf04[['zip', 'est', 'year', 'name']]
df05 = zf05[['zip', 'est', 'year', 'name']]
df06 = zf06[['zip', 'est', 'year', 'name']]
df07 = zf07[['zip', 'est', 'year', 'name']]
df08 = zf08[['zip', 'est', 'year', 'name']]
df09 = zf09[['zip', 'est', 'year', 'name']]
df10 = zf10[['zip', 'est', 'year', 'name']]
df11 = zf11[['zip', 'est', 'year', 'name']]
df12 = zf12[['zip', 'est', 'year', 'name']]
df13 = zf13[['zip', 'est', 'year', 'name']]

NameError: name 'zf94' is not defined

In [6]:
#creating dataframe with all years
bigdata = pd.concat((df94, df95, df96, df97, df98, df99, df00, df01, df02, df03, df04, df05, df06, df07, df08, df09, df10, df11, df12, df13), axis = 0)
bigdata.shape

(789999, 4)

In [7]:
#reading in shapefile
shapeData = gp.read_file('/Users/mariaortiz/PUI2015_mortiz/HW10/nycpolygons.geojson')
#making column to merge on Zip later on
shapeData['ZIP'] = shapeData.postalCode.astype(int)
#array of nyczips
nyczips = shapeData.postalCode.astype(int)

In [8]:
#making df for all data, by zip code and by year
dfall = bigdata.loc[bigdata['zip'].isin(nyczips)]

###Awesome! 

####oh...wait a minute...
Now we realized that the data needed to be put with 219 rows (219 zips) x 20 columns (20 years) with the value of est (# of establishments)

In [9]:
sonice = dfall[['zip', 'year', 'est']].groupby(['zip', 'year']).sum().unstack()
sonice.reset_index().head()
sonice.head()
#sonice.shape

/Users/mariaortiz/anaconda/lib/python2.7/site-packages/pandas/core/index.py:4072: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return np.sum(name == np.asarray(self.names)) > 1


est                                                               \
year   1994  1995  1996  1997  1998  1999   2000  2001  2002  2003  2004   
zip                                                                        
10001  6999  7088  7075  7154  7254  7233  14789  7188  7141  7084  7370   
10002  1784  1819  1919  2049  2003  2085   4968  2543  2269  2402  2551   
10003  3256  3377  3417  3535  3649  3659   7754  3602  3616  3704  3776   
10004  1115  1124  1133  1173  1176  1233   2670  1247  1253  1252  1283   
10005  1076  1109  1216  1227  1270  1242   2578  1249  1244  1200  1226   

                                                       
year   2005  2006  2007  2009  2010  2011  2012  2013  
zip                                                    
10001  7312  7415  7549  7305  7241  7227  7235  7273  
10002  2682  2686  2685  2775  2872  2912  2954  2988  
10003  3867  3909  4049  4099  4113  4159  4214  4277  
10004  1260  1311  1370  1379  1398  1404  1442  1479  
10005  1214  1259  1273  1274  1266  1301  1273  1296

In [10]:
#turning the df into an array
final = sonice.values
final

array([[ 6999.,  7088.,  7075., ...,  7227.,  7235.,  7273.],
       [ 1784.,  1819.,  1919., ...,  2912.,  2954.,  2988.],
       [ 3256.,  3377.,  3417., ...,  4159.,  4214.,  4277.],
       ..., 
       [   95.,    90.,    88., ...,   131.,   129.,   127.],
       [  294.,   285.,   287., ...,   326.,   326.,   327.],
       [   33.,    32.,    34., ...,    46.,    49.,    40.]])

##Standardizing time series: subtracting the mean and dividing each time series (separately) by its standard deviation

In [11]:
import numpy as np

yes = []
for i in final:
    rowstd = np.std(i)
    rowmean = np.mean(i)
    i = (i - rowmean)/rowstd
    yes.append(i)

yes2 = np.asarray(yes)
yes2.shape

#yes3 = pd.DataFrame(yes2)
#yes3.head()
#I only have 19 years...should be 20! will try to fix later

(244, 19)

##K means 2 clustering

In [ ]:
#my kernel dies every time...
nk = 3
#cluster_centroids, closest_centroids = kmeans2(yes2, nk, iter=1)
cluster_centroids, closest_centroids = kmeans2(yes2, nk, iter=1, minit = 'points')
#cluster_centroids, closest_centroids = kmeans((yes), nk, iter=20, thresh=1e-05) 

In [17]:
array=[1, 2, 3, 4, 4, 2]
array2 = [2,3,6,3,6,7]
test = concatenate([array, array2])

In [ ]:
#plotting data for all years
plt.figure(figsize=(10,4))
#plt.xlim(1994,2012)

for i in range(len(yes)):
    pl.plot(yes2[i])
    pl.xlabel("years", fontsize=18)
    pl.ylim(-3,3)
    #pl.xlim(1994,2012)

In [ ]:
print(cluster_centroids.shape)
dates = list(range(1994,2013))
cluster_centroidsDF = pd.DataFrame(cluster_centroids, columns = dates)

In [ ]:
#plotting centroids
for i in range(0,3):
    pl.plot(cluster_centroidsDF.loc[i,dates])

In [ ]:
#clusteryes = yes2.join(pd.Series(closest_centroids, name = 'Cluster Number'))
cluster_yes = yes2.join(pd.Series(closest_centroids, name = 'Cluster Number'))

##DBSCAN clustering

In [ ]:
badindex = []
for i,est in enumerate(yes):
    if np.isnan(est).any():
        badindex.append(i)
        
yesnonan = np.delete(yes, badindex, 0)

 

In [ ]:
from sklearn.cluster import DBSCAN
db = DBSCAN(eps=5.5, min_samples=5).fit(yesnonan)
labels = (db.labels_).astype(int)
#print labels
num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
clusters = pd.Series([yesnonan[labels == i] for i in xrange(num_clusters)])
print('Number of clusters: %d' % num_clusters)

In [ ]:
cluster_myData2 = yesnonan.join(pd.Series(labels, name = 'DBSCAN Cluster Number'))


##Kmeans clustering

In [ ]:
nc = 5
clustering = sklearn.cluster.KMeans(n_clusters=nc).fit(clean_est)
clustering

In [ ]:
#plotting data for all years
plt.figure(figsize=(10,4))
#plt.xlim(1994,2012)

for i in range(len(clean_est)):
    pl.plot(clean_est[i])
    pl.xlabel("years", fontsize=18)
    pl.ylim(-3,3)
    #pl.xlim(1994,2012)

In [ ]:
#opening map data
json_data=open('/Users/mariaortiz/PUI2015_mortiz/HW10/nyc-zip-code-tabulation-areas-polygons.geojson').read()

data2 = json.loads(json_data)
pl.plot(data2)

In [ ]:
import mplleaflet
#mplleaflet.display(crs=data.crs)

##Plotting on map